# Fecha de entrega: 23/07/2022 (corresponde a clase 15)

# Base de datos:

# Diccionario

**Base de datos = BTC_final**

- **Date**: fecha en YYYY-MM-DD (datetime64 [ns])
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)

# Librerías

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
from scipy.stats import norm
from scipy import stats
import talib as ta
from talib import abstract

import warnings
warnings.filterwarnings('once')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Data acquisition

In [2]:
def gdriveColabPath(sharing_url):
  file_id=sharing_url.split('/')[-2]
  dwn_url='https://drive.google.com/uc?id=' + file_id
  return dwn_url

## Adquiriendo base de dato BTC_final

In [3]:
sharing_url = "https://drive.google.com/file/d/1M38n5So0-6r_Q0vhs1i5nIW-cpjgDlww/view?usp=sharing"

In [4]:
dwn_url=gdriveColabPath(sharing_url)
BTC_df =pd.read_csv(dwn_url, sep=";", decimal=".")
BTC_df.head()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target
0,2010-07-18,0.1,0.0,0.1,0.1,80.0,0.0,0.0
1,2010-07-19,0.1,0.1,0.1,0.1,570.0,0.0,0.0
2,2010-07-20,0.1,0.1,0.1,0.1,260.0,0.0,0.0
3,2010-07-21,0.1,0.1,0.1,0.1,580.0,0.0,0.0
4,2010-07-22,0.1,0.1,0.1,0.1,2160.0,0.0,0.0


# Data wrangling

## Etapa de estructuración

Se utilizará la librería talib para calcular los valores de distintos indicadores de series de tiempo: indicadores de tendencia, momento, ciclicidad, patrones, volumen, volatilidad, etc.

* Página web de documentación de la librería: https://mrjbq7.github.io/ta-lib/doc_index.html

### Overlap Studies Functions

#### Media Movil Simple (SMA) (7, 15, 30, 50, 100 y 200 días)

* Información (EN) = https://www.investopedia.com/terms/s/sma.asp

Calculando la Media Móvil Simple (SMA) -> se utiliza ta.SMA (data, timeperiod=)

Se agregan los valores como nuevas columnas

In [5]:
BTC_df_SMA = BTC_df.copy()

In [6]:
BTC_df_SMA ["SMA_7d"] = ta.SMA (BTC_df_SMA ["Price"], timeperiod=7)

In [7]:
BTC_df_SMA ["SMA_15d"] = ta.SMA (BTC_df_SMA ["Price"], timeperiod=15)

In [8]:
BTC_df_SMA ["SMA_30d"] = ta.SMA (BTC_df_SMA ["Price"], timeperiod=30)

In [9]:
BTC_df_SMA ["SMA_50d"] = ta.SMA (BTC_df_SMA ["Price"], timeperiod=50)

In [10]:
BTC_df_SMA ["SMA_100d"] = ta.SMA (BTC_df_SMA ["Price"], timeperiod=100)

In [11]:
BTC_df_SMA ["SMA_200d"] = ta.SMA (BTC_df_SMA ["Price"], timeperiod=200)

In [12]:
BTC_df_SMA.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,SMA_7d,SMA_15d,SMA_30d,SMA_50d,SMA_100d,SMA_200d
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20430.357143,20491.973333,20460.546667,23600.324,29655.396,35689.4685
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20421.528571,20594.800000,20471.913333,23435.494,29440.502,35561.0880
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,20787.614286,20809.193333,20589.890000,23296.706,29238.090,35438.1005
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,21370.371429,21022.153333,20684.323333,23130.444,29050.812,35324.0540
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,21784.528571,21218.720000,20770.216667,22957.558,28887.333,35201.1095


Estrategia de trading con SMA

* Cuando el valor de la columna "Price" supera a la Media Móvil Simple se compra ("Compra"). Se entiende que la tendencia sube.
* Cuando el valor de la columna "Price" cae por debajo de la Media Móvil Simple se vende ("Venta"). Se entiende que la tendencia baja.
* El resto de los valores adquieren valor "Mantener_posicion". Se entiende que no hay cambio de tendencia.

In [13]:
# Se realiza un back up
BTC_df_SMA_ET = BTC_df_SMA.copy()

In [14]:
BTC_df_SMA_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'SMA_7d', 'SMA_15d', 'SMA_30d', 'SMA_50d', 'SMA_100d',
       'SMA_200d'],
      dtype='object')

In [15]:
SMA_Columnas = ["SMA_7d", "SMA_15d", "SMA_30d", "SMA_50d", "SMA_100d","SMA_200d"]

In [16]:
# Se ejecuta la estrategia y se crean las columnas con los resultados
SMA_ET_df = pd.DataFrame()
SMA_ET_df

for col in SMA_Columnas:
    
    SMA_ET = []

    x=0
    while x < len (BTC_df_SMA_ET ["Price"]): 
        
        if x==0:
            SMA_ET.append ("Mantener_posicion")
        
        elif (BTC_df_SMA_ET ["Price"] [x-1] < BTC_df_SMA_ET [col] [x-1]) and (BTC_df_SMA_ET ["Price"] [x] > BTC_df_SMA_ET [col] [x]):
            SMA_ET.append ("Comprar")
            
        elif (BTC_df_SMA_ET ["Price"] [x-1] > BTC_df_SMA_ET [col] [x-1]) and (BTC_df_SMA_ET ["Price"] [x] < BTC_df_SMA_ET [col] [x]):
            SMA_ET.append ("Vender")
            
        else:
            SMA_ET.append ("Mantener_posicion")
            
        
        x+=1
    
    SMA_ET_df [col] = pd.Series(SMA_ET)
    
    SMA_ET.clear()

In [17]:
SMA_ET_df.tail()

,SMA_7d,SMA_15d,SMA_30d,SMA_50d,SMA_100d,SMA_200d
4381,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4383,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4384,Mantener_posicion,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion
4385,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


In [18]:
# Se renombra la columna por comodidad en su tratamiento posterior
for Col in SMA_Columnas:
    SMA_ET_df.rename(columns={""+Col+"": ""+Col+"_ET"}, inplace=True)

In [19]:
SMA_ET_df.tail()

,SMA_7d_ET,SMA_15d_ET,SMA_30d_ET,SMA_50d_ET,SMA_100d_ET,SMA_200d_ET
4381,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4383,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4384,Mantener_posicion,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion
4385,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_SMA_ET (ya posee los valores de las medias móviles)

In [20]:
BTC_SMA_ET_df = pd.concat ([BTC_df_SMA_ET, SMA_ET_df], axis=1)

In [21]:
BTC_SMA_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,SMA_7d,SMA_15d,SMA_30d,SMA_50d,SMA_100d,SMA_200d,SMA_7d_ET,SMA_15d_ET,SMA_30d_ET,SMA_50d_ET,SMA_100d_ET,SMA_200d_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20430.357143,20491.973333,20460.546667,23600.324,29655.396,35689.4685,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20421.528571,20594.800000,20471.913333,23435.494,29440.502,35561.0880,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,20787.614286,20809.193333,20589.890000,23296.706,29238.090,35438.1005,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,21370.371429,21022.153333,20684.323333,23130.444,29050.812,35324.0540,Mantener_posicion,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,21784.528571,21218.720000,20770.216667,22957.558,28887.333,35201.1095,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


#### Media Móvil Exponencial (MME) (7, 15, 30, 50, 100 y 200 días)

* Información (EN): https://www.investopedia.com/terms/e/ema.asp

In [22]:
# Se crea para back up
BTC_df_EMA = BTC_df.copy()

In [23]:
BTC_df_EMA ["EMA_7d"] = ta.EMA (BTC_df_EMA ["Price"], timeperiod=7)

In [24]:
BTC_df_EMA ["EMA_15d"] = ta.EMA (BTC_df_EMA ["Price"], timeperiod=15)

In [25]:
BTC_df_EMA ["EMA_30d"] = ta.EMA (BTC_df_EMA ["Price"], timeperiod=30)

In [26]:
BTC_df_EMA ["EMA_50d"] = ta.EMA (BTC_df_EMA ["Price"], timeperiod=50)

In [27]:
BTC_df_EMA ["EMA_100d"] = ta.EMA (BTC_df_EMA ["Price"], timeperiod=100)

In [28]:
BTC_df_EMA ["EMA_200d"] = ta.EMA (BTC_df_EMA ["Price"], timeperiod=200)

In [29]:
BTC_df_EMA.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,EMA_7d,EMA_15d,EMA_30d,EMA_50d,EMA_100d,EMA_200d
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20679.126620,20677.015769,21599.199372,23575.445072,28195.025858,33931.069335
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20705.744965,20690.588798,21546.709090,23466.039383,28048.304554,33800.268645
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,21160.758724,20919.990198,21609.876245,23429.167250,27938.948029,33688.084877
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,21723.119043,21231.266424,21726.026165,23428.423436,27849.269850,33585.817366
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,22079.614282,21470.995621,21817.837380,23417.469576,27756.197179,33481.969432


Estrategia de trading


* Cuando el valor de la columna "Price" supera a la Media Móvil Exponencial se compra ("Compra"). Se entiende que la tendencia sube.
* Cuando el valor de la columna "Price" cae por debajo de la Media Móvil Exponencial se vende ("Venta"). Se entiende que la tendencia baja.
* El resto de los valores adquieren valor "Mantener_posicion". Se entiende que la tendencia se mantiene.

In [30]:
# Se crea para back up
BTC_df_EMA_ET = BTC_df_EMA.copy()

In [31]:
BTC_df_EMA_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'EMA_7d', 'EMA_15d', 'EMA_30d', 'EMA_50d', 'EMA_100d',
       'EMA_200d'],
      dtype='object')

In [32]:
EMA_Columnas = ['EMA_7d', 'EMA_15d', 'EMA_30d', 'EMA_50d', 'EMA_100d', 'EMA_200d'] 

In [33]:
# Se aplica la estrategia de trading
EMA_ET_df = pd.DataFrame()
EMA_ET_df

for col in EMA_Columnas:
    
    EMA_ET = []

    x=0
    while x < len (BTC_df_EMA_ET ["Price"]): 
        
        if x==0:
            EMA_ET.append ("Mantener_posicion")
        
        elif (BTC_df_EMA_ET ["Price"] [x-1] < BTC_df_EMA_ET [col] [x-1]) and (BTC_df_EMA_ET ["Price"] [x] > BTC_df_EMA_ET [col] [x]):
            EMA_ET.append ("Comprar")
            
        elif (BTC_df_EMA_ET ["Price"] [x-1] > BTC_df_EMA_ET [col] [x-1]) and (BTC_df_EMA_ET ["Price"] [x] < BTC_df_EMA_ET [col] [x]):
            EMA_ET.append ("Vender")
            
        else:
            EMA_ET.append ("Mantener_posicion")
            
        
        x+=1
    
    EMA_ET_df [col] = pd.Series(EMA_ET)
    
    EMA_ET.clear()

In [34]:
EMA_ET_df.tail()

,EMA_7d,EMA_15d,EMA_30d,EMA_50d,EMA_100d,EMA_200d
4381,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4383,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion,Mantener_posicion
4384,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4385,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


In [35]:
# Se renombran las columnas para su posterior tratamiento
for Col in EMA_Columnas:
    EMA_ET_df.rename(columns={""+Col+"": ""+Col+"_ET"}, inplace=True)

In [36]:
EMA_ET_df.tail()

,EMA_7d_ET,EMA_15d_ET,EMA_30d_ET,EMA_50d_ET,EMA_100d_ET,EMA_200d_ET
4381,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4383,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion,Mantener_posicion
4384,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4385,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_EMA_ET (ya posee los valores de las medias móviles)

In [37]:
BTC_EMA_ET_df = pd.concat ([BTC_df_EMA_ET, EMA_ET_df], axis=1)

In [38]:
BTC_EMA_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,EMA_7d,EMA_15d,EMA_30d,EMA_50d,EMA_100d,EMA_200d,EMA_7d_ET,EMA_15d_ET,EMA_30d_ET,EMA_50d_ET,EMA_100d_ET,EMA_200d_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20679.126620,20677.015769,21599.199372,23575.445072,28195.025858,33931.069335,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20705.744965,20690.588798,21546.709090,23466.039383,28048.304554,33800.268645,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,21160.758724,20919.990198,21609.876245,23429.167250,27938.948029,33688.084877,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,21723.119043,21231.266424,21726.026165,23428.423436,27849.269850,33585.817366,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,22079.614282,21470.995621,21817.837380,23417.469576,27756.197179,33481.969432,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


#### HT_TRENDLINE - Hilbert Transform - Instantaneous Trendline

* Información (EN): John F. Ehlers, Rocket Science for Traders. *Trading with signal processing methods*. Source code in EasyLanguage.
* Más información (EN): https://zorro-project.com/manual/en/ta.htm

In [39]:
# Se realiza un back up
BTC_df_HT_TRENDLINE = BTC_df.copy()

In [40]:
# Se calcula el indicador
BTC_df_HT_TRENDLINE ["HTTL"] = ta.HT_TRENDLINE (BTC_df_HT_TRENDLINE ["Price"])

In [41]:
BTC_df_HT_TRENDLINE.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,HTTL
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20423.078104
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20403.832160
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,20435.796762
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,20547.109474
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,20712.814316


Estrategia de trading con HT_Trendline

* Cuando el valor de la columna "Price" supera a la HT_Trendline se compra ("Compra"). Se entiende que la tendencia sube.
* Cuando el valor de la columna "Price" cae por debajo de la HT_Trendline se vende ("Venta"). Se entiende que la tendencia baja.
* El resto de los valores adquieren valor "Mantener_posicion". Se entiende que la tendencia se mantiene.

In [42]:
# Se crea un back up
BTC_df_HT_ET = BTC_df_HT_TRENDLINE.copy()

In [43]:
BTC_df_HT_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'HTTL'],
      dtype='object')

In [44]:
HT_Columnas = ['HTTL']

In [45]:
# Se calcula la estrategia de trading
HT_ET_df = pd.DataFrame()
HT_ET_df

for col in HT_Columnas:
    
    HT_ET = []

    x=0
    while x < len (BTC_df_HT_ET ["Price"]): 
        
        if x==0:
            HT_ET.append ("Mantener_posicion")
        
        elif (BTC_df_HT_ET ["Price"] [x-1] < BTC_df_HT_ET [col] [x-1]) and (BTC_df_HT_ET ["Price"] [x] > BTC_df_HT_ET [col] [x]):
            HT_ET.append ("Comprar")
            
        elif (BTC_df_HT_ET ["Price"] [x-1] > BTC_df_HT_ET [col] [x-1]) and (BTC_df_HT_ET ["Price"] [x] < BTC_df_HT_ET [col] [x]):
            HT_ET.append ("Vender")
            
        else:
            HT_ET.append ("Mantener_posicion")
            
        
        x+=1
    
    HT_ET_df [col] = pd.Series(HT_ET)
    
    HT_ET.clear()

In [46]:
HT_ET_df.tail()

,HTTL
4381,Mantener_posicion
4382,Mantener_posicion
4383,Mantener_posicion
4384,Mantener_posicion
4385,Mantener_posicion


In [47]:
# Se renombra la columna para posterior tratamiento
HT_ET_df.rename(columns={"HTTL": "HTTL_ET"}, inplace=True)

In [48]:
HT_ET_df.tail()

,HTTL_ET
4381,Mantener_posicion
4382,Mantener_posicion
4383,Mantener_posicion
4384,Mantener_posicion
4385,Mantener_posicion


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_HT_ET (ya posee los valores del indicador).

In [49]:
BTC_HT_ET_df = pd.concat ([BTC_df_HT_ET, HT_ET_df], axis=1) 

In [50]:
BTC_HT_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,HTTL,HTTL_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20423.078104,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20403.832160,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,20435.796762,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,20547.109474,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,20712.814316,Mantener_posicion


#### Bandas de Bollinger

* Información (EN): https://www.investopedia.com/terms/b/bollingerbands.asp

In [51]:
# Se crea un back up
BTC_df_BBANDS = BTC_df.copy()

In [52]:
# Se aplica ta.BBAND que da como resultado la linea más alta, la media y la más baja del indicador de las Bandas de Bollinger
BTC_df_BBANDS ["BBANDS_upper_band"], BTC_df_BBANDS ["BBANDS_middle_band"], BTC_df_BBANDS ["BBANDS_lower_band"]  = ta.BBANDS (BTC_df_BBANDS ["Price"])

In [53]:
BTC_df_BBANDS.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,BBANDS_upper_band,BBANDS_middle_band,BBANDS_lower_band
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,21714.292433,20440.38,19166.467567
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,21359.693563,20731.32,20102.946437
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,22585.394742,21186.48,19787.565258
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,23836.570363,21751.32,19666.069637
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,24304.381782,22216.12,20127.858218


Observaciones:
- Por si solo las Bandas de Bollinger no pueden ser consideradas para una estrategia de trading individual.

#### Parabolic SAR

* Información (EN): https://www.investopedia.com/trading/introduction-to-parabolic-sar/#:~:text=The%20parabolic%20SAR%20is%20a,1%EF%BB%BF

In [54]:
# Se crea un back up
BTC_df_PSAR = BTC_df.copy ()

In [55]:
# Se aplica ta.SAR (indicador)
BTC_df_PSAR ["PSAR"] = ta.SAR (BTC_df_PSAR ["High"], BTC_df_PSAR ["Low"], acceleration=0.02, maximum=0.2)

In [56]:
BTC_df_PSAR.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,PSAR
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,22017.409626
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,21894.401241
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,18942.200000
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,19017.654000
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,19207.239840


Estrategia de trading con PSAR

* Cuando el valor de la columna "Price" supera a al PSAR se compra ("Compra"). Se entiende que la tendencia sube.
* Cuando el valor de la columna "Price" cae por debajo del PSAR se vende ("Venta"). Se entiende que la tendencia baja.
* El resto de los valores adquieren valor "Mantener_posicion". Se entiende que la tendencia se mantiene.

In [57]:
BTC_df_PSAR_ET = BTC_df_PSAR.copy()

In [58]:
BTC_df_PSAR_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'PSAR'],
      dtype='object')

In [59]:
PSAR_Columnas = ['PSAR'] 

In [60]:
# Se aplica la estrategia de trading
PSAR_ET_df = pd.DataFrame()
PSAR_ET_df

for col in PSAR_Columnas:
    
    PSAR_ET = []

    x=0
    while x < len (BTC_df_PSAR_ET ["Price"]): 
        
        if x==0:
            PSAR_ET.append ("Mantener_posicion")
        
        elif (BTC_df_PSAR_ET ["Price"] [x-1] < BTC_df_PSAR_ET [col] [x-1]) and (BTC_df_PSAR_ET ["Price"] [x] > BTC_df_PSAR_ET [col] [x]):
            PSAR_ET.append ("Comprar")
            
        elif (BTC_df_PSAR_ET ["Price"] [x-1] > BTC_df_PSAR_ET [col] [x-1]) and (BTC_df_PSAR_ET ["Price"] [x] < BTC_df_PSAR_ET [col] [x]):
            PSAR_ET.append ("Vender")
            
        else:
            PSAR_ET.append ("Mantener_posicion")
            
        
        x+=1
    
    PSAR_ET_df [col] = pd.Series(PSAR_ET)
    
    PSAR_ET.clear()

In [61]:
PSAR_ET_df.tail()

,PSAR
4381,Mantener_posicion
4382,Mantener_posicion
4383,Comprar
4384,Mantener_posicion
4385,Mantener_posicion


In [62]:
# Se renombra la columna por comodidad en su tratamiento posterior
PSAR_ET_df.rename(columns={"PSAR": "PSAR_ET"}, inplace=True)

In [63]:
PSAR_ET_df.tail()

,PSAR_ET
4381,Mantener_posicion
4382,Mantener_posicion
4383,Comprar
4384,Mantener_posicion
4385,Mantener_posicion


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_PSAR_ET (ya posee los valores del indicador).

In [64]:
BTC_PSAR_ET_df = pd.concat ([BTC_df_PSAR_ET, PSAR_ET_df], axis=1)

In [65]:
BTC_PSAR_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,PSAR,PSAR_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,22017.409626,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,21894.401241,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,18942.200000,Comprar
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,19017.654000,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,19207.239840,Mantener_posicion


#### Uniendo los dataframe de indicadores de tendencia a fin de tener una sola base de datos => BTC_ind_trend

Data set sin estrategias de trading

In [66]:
BTC_df_SMA.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'SMA_7d', 'SMA_15d', 'SMA_30d', 'SMA_50d', 'SMA_100d',
       'SMA_200d'],
      dtype='object')

In [67]:
BTC_df_EMA.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'EMA_7d', 'EMA_15d', 'EMA_30d', 'EMA_50d', 'EMA_100d',
       'EMA_200d'],
      dtype='object')

In [68]:
BTC_df_HT_TRENDLINE.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'HTTL'],
      dtype='object')

In [69]:
BTC_df_BBANDS.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'BBANDS_upper_band', 'BBANDS_middle_band',
       'BBANDS_lower_band'],
      dtype='object')

In [70]:
BTC_df_PSAR.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'PSAR'],
      dtype='object')

In [71]:
BTC_ind_trend = pd.DataFrame ({'Date':BTC_df_SMA ["Date"], 'Price':BTC_df_SMA ["Price"], 
                              'Open': BTC_df_SMA ["Open"], 'High': BTC_df_SMA ["High"], 'Low': BTC_df_SMA ["Low"], 
                              'Vol.': BTC_df_SMA ["Vol."], 'Percentage_diff': BTC_df_SMA ["Percentage_diff"],'Target': BTC_df_SMA ["Target"], 
                              'SMA_7d': BTC_df_SMA ["SMA_7d"], 'SMA_15d': BTC_df_SMA ["SMA_15d"], 
                              'SMA_30d': BTC_df_SMA ["SMA_30d"], 'SMA_50d': BTC_df_SMA ["SMA_50d"], 
                              'SMA_100d': BTC_df_SMA ["SMA_100d"],'SMA_200d':BTC_df_SMA ["SMA_200d"],
                              'EMA_7d': BTC_df_EMA ["EMA_7d"], 'EMA_15d':BTC_df_EMA ["EMA_15d"],
                              'EMA_30d': BTC_df_EMA ["EMA_30d"], 'EMA_50d': BTC_df_EMA ["EMA_50d"], 
                              'EMA_100d': BTC_df_EMA ["EMA_100d"],'EMA_200d': BTC_df_EMA ["EMA_200d"],
                              'HT_TRENDLINE': BTC_df_HT_TRENDLINE ["HTTL"],
                              "BBANDS_upper_band": BTC_df_BBANDS ["BBANDS_upper_band"],
                              "BBANDS_middle_band": BTC_df_BBANDS ["BBANDS_middle_band"],
                              "BBANDS_lower_band": BTC_df_BBANDS ["BBANDS_lower_band"],
                              'PSAR': BTC_df_PSAR ['PSAR']})

In [72]:
BTC_ind_trend.head()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,SMA_7d,SMA_15d,SMA_30d,SMA_50d,SMA_100d,SMA_200d,EMA_7d,EMA_15d,EMA_30d,EMA_50d,EMA_100d,EMA_200d,HT_TRENDLINE,BBANDS_upper_band,BBANDS_middle_band,BBANDS_lower_band,PSAR
0,2010-07-18,0.1,0.0,0.1,0.1,80.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-07-19,0.1,0.1,0.1,0.1,570.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1
2,2010-07-20,0.1,0.1,0.1,0.1,260.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1
3,2010-07-21,0.1,0.1,0.1,0.1,580.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1
4,2010-07-22,0.1,0.1,0.1,0.1,2160.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.1,0.1,0.1


Observaciones:
- Los valores NaN se debe a que los indicadores de tendencia necesitan valores primeros para poder calcular el resto de los valores.

In [73]:
BTC_ind_trend.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,SMA_7d,SMA_15d,SMA_30d,SMA_50d,SMA_100d,SMA_200d,EMA_7d,EMA_15d,EMA_30d,EMA_50d,EMA_100d,EMA_200d,HT_TRENDLINE,BBANDS_upper_band,BBANDS_middle_band,BBANDS_lower_band,PSAR
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20430.357143,20491.973333,20460.546667,23600.324,29655.396,35689.4685,20679.126620,20677.015769,21599.199372,23575.445072,28195.025858,33931.069335,20423.078104,21714.292433,20440.38,19166.467567,22017.409626
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20421.528571,20594.800000,20471.913333,23435.494,29440.502,35561.0880,20705.744965,20690.588798,21546.709090,23466.039383,28048.304554,33800.268645,20403.832160,21359.693563,20731.32,20102.946437,21894.401241
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,20787.614286,20809.193333,20589.890000,23296.706,29238.090,35438.1005,21160.758724,20919.990198,21609.876245,23429.167250,27938.948029,33688.084877,20435.796762,22585.394742,21186.48,19787.565258,18942.200000
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,21370.371429,21022.153333,20684.323333,23130.444,29050.812,35324.0540,21723.119043,21231.266424,21726.026165,23428.423436,27849.269850,33585.817366,20547.109474,23836.570363,21751.32,19666.069637,19017.654000
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,21784.528571,21218.720000,20770.216667,22957.558,28887.333,35201.1095,22079.614282,21470.995621,21817.837380,23417.469576,27756.197179,33481.969432,20712.814316,24304.381782,22216.12,20127.858218,19207.239840


In [74]:
BTC_ind_trend.dtypes

Date                   object
Price                 float64
Open                  float64
High                  float64
Low                   float64
Vol.                  float64
Percentage_diff       float64
Target                float64
SMA_7d                float64
SMA_15d               float64
SMA_30d               float64
SMA_50d               float64
SMA_100d              float64
SMA_200d              float64
EMA_7d                float64
EMA_15d               float64
EMA_30d               float64
EMA_50d               float64
EMA_100d              float64
EMA_200d              float64
HT_TRENDLINE          float64
BBANDS_upper_band     float64
BBANDS_middle_band    float64
BBANDS_lower_band     float64
PSAR                  float64
dtype: object

In [77]:
BTC_ind_trend.to_csv(r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_trend.csv", index=False, sep=";")

Data set con estrategia de trading

In [78]:
BTC_ind_trend_ET = BTC_ind_trend.copy()

In [79]:
SMA_ET_df.columns

Index(['SMA_7d_ET', 'SMA_15d_ET', 'SMA_30d_ET', 'SMA_50d_ET', 'SMA_100d_ET',
       'SMA_200d_ET'],
      dtype='object')

In [80]:
EMA_ET_df.columns

Index(['EMA_7d_ET', 'EMA_15d_ET', 'EMA_30d_ET', 'EMA_50d_ET', 'EMA_100d_ET',
       'EMA_200d_ET'],
      dtype='object')

In [81]:
HT_ET_df.columns

Index(['HTTL_ET'], dtype='object')

In [82]:
PSAR_ET_df.columns

Index(['PSAR_ET'], dtype='object')

In [83]:
BTC_ind_trend_ET = pd.concat ([BTC_ind_trend, SMA_ET_df, EMA_ET_df, HT_ET_df, PSAR_ET_df], axis=1)

In [84]:
BTC_ind_trend_ET.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,SMA_7d,SMA_15d,SMA_30d,SMA_50d,SMA_100d,SMA_200d,EMA_7d,EMA_15d,EMA_30d,EMA_50d,EMA_100d,EMA_200d,HT_TRENDLINE,BBANDS_upper_band,BBANDS_middle_band,BBANDS_lower_band,PSAR,SMA_7d_ET,SMA_15d_ET,SMA_30d_ET,SMA_50d_ET,SMA_100d_ET,SMA_200d_ET,EMA_7d_ET,EMA_15d_ET,EMA_30d_ET,EMA_50d_ET,EMA_100d_ET,EMA_200d_ET,HTTL_ET,PSAR_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20430.357143,20491.973333,20460.546667,23600.324,29655.396,35689.4685,20679.126620,20677.015769,21599.199372,23575.445072,28195.025858,33931.069335,20423.078104,21714.292433,20440.38,19166.467567,22017.409626,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,20421.528571,20594.800000,20471.913333,23435.494,29440.502,35561.0880,20705.744965,20690.588798,21546.709090,23466.039383,28048.304554,33800.268645,20403.832160,21359.693563,20731.32,20102.946437,21894.401241,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,20787.614286,20809.193333,20589.890000,23296.706,29238.090,35438.1005,21160.758724,20919.990198,21609.876245,23429.167250,27938.948029,33688.084877,20435.796762,22585.394742,21186.48,19787.565258,18942.200000,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Comprar
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,21370.371429,21022.153333,20684.323333,23130.444,29050.812,35324.0540,21723.119043,21231.266424,21726.026165,23428.423436,27849.269850,33585.817366,20547.109474,23836.570363,21751.32,19666.069637,19017.654000,Mantener_posicion,Mantener_posicion,Mantener_posicion,Comprar,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,21784.528571,21218.720000,20770.216667,22957.558,28887.333,35201.1095,22079.614282,21470.995621,21817.837380,23417.469576,27756.197179,33481.969432,20712.814316,24304.381782,22216.12,20127.858218,19207.239840,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


In [85]:
BTC_ind_trend_ET.to_csv(r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_trend_ET.csv", index=False, sep=";")

### Momentum Indicator Functions

#### ADX (Average Directional Movement Index)

* Información (EN): https://www.investopedia.com/terms/a/adx.asp

In [86]:
# Se crea base de dato para back up
BTC_df_ADX = BTC_df.copy ()

In [87]:
# Se aplica ta.ADX (se calcula el ADX)
BTC_df_ADX ["ADX"] = ta.ADX (BTC_df_ADX ["High"], BTC_df_ADX ["Low"], BTC_df_ADX ["Price"], timeperiod=14)

Se calculan valores que conforman el ADX que también pueden tener una relevancia para conocer si un precio subio o bajó

In [88]:
BTC_df_ADX ["ADX_MDI"] = ta.MINUS_DI (BTC_df_ADX ["High"], BTC_df_ADX ["Low"], BTC_df_ADX ["Price"], timeperiod=14)

In [89]:
BTC_df_ADX ["ADX_MDM"] = ta.MINUS_DM (BTC_df_ADX ["High"], BTC_df_ADX ["Low"], timeperiod=14)

In [90]:
BTC_df_ADX ["ADX_DX"] = ta.DX (BTC_df_ADX ["High"], BTC_df_ADX ["Low"], BTC_df_ADX ["Price"], timeperiod=14)

In [91]:
BTC_df_ADX ["ADX_PDI"] = ta.PLUS_DI (BTC_df_ADX ["High"], BTC_df_ADX ["Low"], BTC_df_ADX ["Price"], timeperiod=14)

In [92]:
BTC_df_ADX ["ADX_PDM"] = ta.PLUS_DM (BTC_df_ADX ["High"], BTC_df_ADX ["Low"], timeperiod=14)

In [93]:
BTC_df_ADX.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADX,ADX_MDI,ADX_MDM,ADX_DX,ADX_PDI,ADX_PDM
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,27.268083,24.194644,3952.577867,6.885001,21.077646,3443.366972
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,25.710035,22.840734,3670.250876,5.455418,20.477541,3290.512188
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,24.545625,20.207578,3408.090099,9.408284,24.404841,4115.975603
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,24.304487,17.742843,3164.655092,21.169702,27.272446,4864.377346
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,24.080574,17.259314,2938.608300,21.169702,26.529215,4516.921821


Estraetgia de trading con ADX

* Cuando ADX_MDI cae por debajo del ADX_PDF puede ser un momento propicio para la compra. Se entiende que hay sobreventa.
* Cuando ADX_MDI supera el ADX_PDF puede ser un momento propicio para la venta. Se entiende que hay sobrecompra.
* En el resto de los casos puede ser un momento propicio para mantener la posición. Hay aumento o reducción de la fuerza tendencial.

In [94]:
# Se crea un back up
BTC_df_ADX_ET = BTC_df_ADX.copy()

In [95]:
BTC_df_ADX_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ADX', 'ADX_MDI', 'ADX_MDM', 'ADX_DX', 'ADX_PDI', 'ADX_PDM'],
      dtype='object')

In [96]:
# Se aplica la estrategia de trading
ADX_ET_df = pd.DataFrame()
ADX_ET_df

ADX_ET = []

x=0
while x < len (BTC_df_ADX_ET ["Price"]): 
        
    if x==0:
        ADX_ET.append ("Mantener_posicion")
        
    elif (BTC_df_ADX_ET ['ADX_MDI'] [x-1] > BTC_df_ADX_ET ['ADX_PDI'] [x-1]) and (BTC_df_ADX_ET ['ADX_MDI'] [x] < BTC_df_ADX_ET ['ADX_PDI'] [x]):
        ADX_ET.append ("Comprar")
            
    elif (BTC_df_ADX_ET ['ADX_MDI'] [x-1] < BTC_df_ADX_ET ['ADX_PDI'] [x-1]) and (BTC_df_ADX_ET ['ADX_MDI'] [x] > BTC_df_ADX_ET ['ADX_PDI'] [x]):
        ADX_ET.append ("Vender")
            
    else:
        ADX_ET.append ("Mantener_posicion")
            
        
    x+=1
    
ADX_ET_df ["ADX_ET"] = pd.Series(ADX_ET)

In [97]:
ADX_ET_df.tail()

,ADX_ET
4381,Mantener_posicion
4382,Mantener_posicion
4383,Comprar
4384,Mantener_posicion
4385,Mantener_posicion


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_ADX_ET (ya posee los valores del indicador).

In [98]:
BTC_ADX_ET_df = pd.concat ([BTC_df_ADX_ET, ADX_ET_df], axis=1)

In [99]:
BTC_ADX_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADX,ADX_MDI,ADX_MDM,ADX_DX,ADX_PDI,ADX_PDM,ADX_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,27.268083,24.194644,3952.577867,6.885001,21.077646,3443.366972,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,25.710035,22.840734,3670.250876,5.455418,20.477541,3290.512188,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,24.545625,20.207578,3408.090099,9.408284,24.404841,4115.975603,Comprar
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,24.304487,17.742843,3164.655092,21.169702,27.272446,4864.377346,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,24.080574,17.259314,2938.608300,21.169702,26.529215,4516.921821,Mantener_posicion


#### ADXR - Average Directional Movement Index Rating

* Información (EN): https://www.linnsoft.com/techind/adxr-avg-directional-movement-rating

In [100]:
# Se crea un back up
BTC_df_ADXR = BTC_df.copy ()

In [101]:
# Se calcula el ADXR
BTC_df_ADXR ["ADXR"] = ta.ADXR (BTC_df_ADXR ["High"], BTC_df_ADXR ["Low"], BTC_df_ADXR ["Price"], timeperiod=14)

In [102]:
BTC_df_ADXR.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADXR
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,36.496257
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,35.321437
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,34.129101
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,33.441982
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,32.144501


Estrategia de trading con ADXR

* Cuando el ADXR supera los 25 pptos puede ser un buen momento para comprar el activo (se entiende que estaría empezando una tendencia alcista).
* Cuando el ADXR cae por debajo de los 25 pptos puede  ser un buen momento para vender el activo (se entiende que estaría empezando una tendencia bajista o una tendencia lateral).
* En el resto de los casos se recomienda mantener la última posición tomada.

In [103]:
BTC_df_ADXR_ET = BTC_df_ADXR.copy()

In [104]:
BTC_df_ADXR_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ADXR'],
      dtype='object')

In [105]:
# Se aplica la estrategia de trading
ADXR_ET_df = pd.DataFrame()
ADXR_ET_df

ADXR_ET = []

x=0
while x < len (BTC_df_ADXR_ET ["Price"]): 
    
    if x<=50: # Porque utiliza 50 registros para calcular el ADXR (se necesita para equiparar con la base datos a la que se concatenará)
        ADXR_ET.append ("Mantener_posicion")
        
    elif BTC_df_ADXR_ET ["ADXR"] [x] >= 25:
        
        if BTC_df_ADXR_ET ["ADXR"] [x-1] >=25:
            ADXR_ET.append ("Mantener_posicion")
            
        elif BTC_df_ADXR_ET ["ADXR"] [x-1] < 25:
            ADXR_ET.append ("Comprar")
    
    elif BTC_df_ADXR_ET ["ADXR"] [x] < 25:
        
        if BTC_df_ADXR_ET ["ADXR"] [x-1] >= 25:
            ADXR_ET.append ("Vender")
            
        if BTC_df_ADXR_ET ["ADXR"] [x-1] < 25:
            ADXR_ET.append ("Mantener_posicion")
            
        
    x+=1
    
ADXR_ET_df ["ADXR_ET"] = pd.Series(ADXR_ET)

In [106]:
ADXR_ET_df.tail()

,ADXR_ET
4381,Mantener_posicion
4382,Mantener_posicion
4383,Mantener_posicion
4384,Mantener_posicion
4385,Mantener_posicion


In [107]:
ADXR_ET_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ADXR_ET  4386 non-null   object
dtypes: object(1)
memory usage: 34.4+ KB


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_ADXR_ET (ya posee los valores del indicador).

In [108]:
BTC_ADXR_ET_df = pd.concat ([BTC_df_ADXR_ET, ADXR_ET_df], axis=1)

In [109]:
BTC_ADXR_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADXR,ADXR_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,36.496257,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,35.321437,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,34.129101,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,33.441982,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,32.144501,Mantener_posicion


#### AROON - Aroon

* Información (EN): https://www.investopedia.com/terms/a/aroon.asp

In [110]:
# Se crea un back up
BTC_df_Aroon = BTC_df.copy ()

In [111]:
# Se calcula las dos variables principales del indicador Aroon
BTC_df_Aroon ["Aroondown"], BTC_df_Aroon ["Aroonup"] = ta.AROON (BTC_df_Aroon ["High"], BTC_df_Aroon ["Low"], timeperiod=14)

In [112]:
BTC_df_Aroon.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,Aroondown,Aroonup
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,7.142857,42.857143
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,0.000000,35.714286
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,64.285714,100.000000
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,57.142857,100.000000
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,50.000000,92.857143


In [113]:
# Se calcula el indicador Aroon
BTC_df_Aroon ["Aroon_Oscillator"] = ta.AROONOSC (BTC_df_Aroon ["High"], BTC_df_Aroon ["Low"], timeperiod=14)

In [114]:
BTC_df_Aroon.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,Aroondown,Aroonup,Aroon_Oscillator
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,7.142857,42.857143,35.714286
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,0.000000,35.714286,35.714286
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,64.285714,100.000000,35.714286
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,57.142857,100.000000,42.857143
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,50.000000,92.857143,42.857143


Observaciones:
- Por si solo no suele utilizarse como estrategia de trading. Suele confirmar sobrecompras/ventas y puntos de inflexión.

#### MACD - Moving Average Convergence/Divergence

* Información (EN): https://www.investopedia.com/terms/m/macd.asp

In [115]:
# Se realiza un back up
BTC_df_MACD = BTC_df.copy()

In [116]:
# Se calcular los diferentes componentes del indicador MACD
BTC_df_MACD ["MACD"], BTC_df_MACD ["MACDSIGNAL"], BTC_df_MACD ["MACDHIST"] = ta.MACD(BTC_df_MACD ["Price"], fastperiod=12, slowperiod=26, signalperiod=9)

In [117]:
BTC_df_MACD.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,MACD,MACDSIGNAL,MACDHIST
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,-636.592931,-960.502376,323.909445
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,-577.232483,-883.848398,306.615914
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,-385.327489,-784.144216,398.816727
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,-160.032949,-659.321963,499.289013
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,-2.524886,-527.962547,525.437661


Estrategia de trading de MACD

* Se entiende que cuando el MACDSIGANL cae por debajo del MACD puede ser un buen momento para comprar (Sobreventa)
* Se entiende que cuando el MACDSIGANL cae por debajo del MACD puede ser un buen momento para vender (Sobrecompra).
* En ninguno de los casos anteriores se recomienda mantener la última posición tomada (No parece haber cambio tendencial).

In [118]:
BTC_df_MACD_ET = BTC_df_MACD.copy()

In [119]:
BTC_df_MACD_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'MACD', 'MACDSIGNAL', 'MACDHIST'],
      dtype='object')

In [120]:
# Se aplica la estrategia de trading
MACD_ET_df = pd.DataFrame()
MACD_ET_df

MACD_ET = []

x=0
while x < len (BTC_df_MACD_ET ["Price"]): 
        
    if x==0:
        MACD_ET.append ("Mantener_posicion")
        
    elif (BTC_df_MACD_ET ['MACDSIGNAL'] [x-1] > BTC_df_MACD_ET ['MACD'] [x-1]) and (BTC_df_MACD_ET ['MACDSIGNAL'] [x] < BTC_df_MACD_ET ['MACD'] [x]):
        MACD_ET.append ("Comprar")
            
    elif (BTC_df_MACD_ET ['MACDSIGNAL'] [x-1] < BTC_df_MACD_ET ['MACD'] [x-1]) and (BTC_df_MACD_ET ['MACDSIGNAL'] [x] > BTC_df_MACD_ET ['MACD'] [x]):
        MACD_ET.append ("Vender")
            
    else:
        MACD_ET.append ("Mantener_posicion")
            
        
    x+=1
    
MACD_ET_df ["MACD_ET"] = pd.Series(MACD_ET)

In [121]:
MACD_ET_df.tail()

,MACD_ET
4381,Mantener_posicion
4382,Mantener_posicion
4383,Mantener_posicion
4384,Mantener_posicion
4385,Mantener_posicion


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_MACD_ET (ya posee los valores del indicador).

In [122]:
BTC_MACD_ET_df = pd.concat ([BTC_df_MACD_ET, MACD_ET_df], axis=1)

In [123]:
BTC_MACD_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,MACD,MACDSIGNAL,MACDHIST,MACD_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,-636.592931,-960.502376,323.909445,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,-577.232483,-883.848398,306.615914,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,-385.327489,-784.144216,398.816727,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,-160.032949,-659.321963,499.289013,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,-2.524886,-527.962547,525.437661,Mantener_posicion


#### Momentum (MOM)

* Información (EN): https://www.investopedia.com/investing/momentum-and-relative-strength-index/

In [124]:
# Se crea un back up
BTC_df_MOM = BTC_df.copy()

In [125]:
# Se calcula el indicador
BTC_df_MOM ["MOM"] = ta.MOM (BTC_df_MOM ["Price"], timeperiod=10)

In [126]:
BTC_df_MOM.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,MOM
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,648.8
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,-852.2
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,914.6
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,1822.7
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,2301.7


Observaciones:
- No se utilizará para calcular una estrategia de trading.

#### ROC - Rate of change

* Información (EN): https://www.investopedia.com/terms/r/rateofchange.asp

In [127]:
# Se crea back up
BTC_df_ROC = BTC_df.copy()

In [128]:
# Se calcula el indicador
BTC_df_ROC ["ROC"] = ta.ROCP (BTC_df_ROC ["Price"], timeperiod=10)

In [129]:
BTC_df_ROC.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ROC
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,0.031555
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,-0.039385
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,0.042321
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,0.084433
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,0.110407


Estrategia de trading usando ROC

* Cuando el ROC supera el nivel 0 se recomienda comprar (sobreventa).
* Cuando el ROC cae por debajo del nivel 0 se recomienda vender (sobrecompra).
* En el resto de la posiciones se mantiene la última decisión tomada (sin cambios).

In [130]:
BTC_df_ROC_ET = BTC_df_ROC.copy()

In [131]:
BTC_df_ROC_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ROC'],
      dtype='object')

In [132]:
# Se aplica estrategia de trading
ROC_ET_df = pd.DataFrame()
ROC_ET_df

ROC_ET = []

x=0
while x < len (BTC_df_ROC_ET ["Price"]): 
    
    if x<11: # Necesita 10 valroes para empezar el proceso (por defecto del uso común del indicador)
        ROC_ET.append ("Mantener_posicion")
        
    elif BTC_df_ROC_ET ["ROC"] [x] >= 0:
        
        if BTC_df_ROC_ET ["ROC"] [x-1] >=0:
            ROC_ET.append ("Mantener_posicion")
            
        elif BTC_df_ROC_ET ["ROC"] [x-1] < 0:
            ROC_ET.append ("Comprar")
    
    elif BTC_df_ROC_ET ["ROC"] [x] < 0:
        
        if BTC_df_ROC_ET ["ROC"] [x-1] >= 0:
            ROC_ET.append ("Vender")
            
        if BTC_df_ROC_ET ["ROC"] [x-1] < 0:
            ROC_ET.append ("Mantener_posicion")
            
        
    x+=1
    
ROC_ET_df ["ROC_ET"] = pd.Series(ROC_ET)

In [133]:
ROC_ET_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ROC_ET  4386 non-null   object
dtypes: object(1)
memory usage: 34.4+ KB


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_ROC_ET (ya posee los valores del indicador).

In [134]:
BTC_ROC_ET_df = pd.concat ([BTC_df_ROC_ET, ROC_ET_df], axis=1)

In [135]:
BTC_ROC_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ROC,ROC_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,0.031555,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,-0.039385,Vender
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,0.042321,Comprar
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,0.084433,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,0.110407,Mantener_posicion


#### Relative Strength Index (RSI) & Stochastic Relative Strength Index (STOCHRSI)

* Información RSI (EN): https://www.investopedia.com/terms/r/rsi.asp
* Información STOCHRSI (EN): https://www.investopedia.com/terms/s/stochrsi.asp

In [136]:
# Se crea un back up
BTC_df_RSI = BTC_df.copy()

In [137]:
# Se calcula RSI
BTC_df_RSI ["RSI"] = ta.RSI (BTC_df_RSI ["Price"], timeperiod=10)

In [138]:
# Se calcula el indicador estocástico rapido según dos versiones k y d
BTC_df_RSI ["STOCHRSI_fastk"], BTC_df_RSI ["STOCHRSI_fastd"]  = ta.STOCHRSI (BTC_df_RSI ["Price"], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)

In [139]:
BTC_df_RSI.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,RSI,STOCHRSI_fastk,STOCHRSI_fastd
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,54.341002,100.000000,100.000000
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,49.615550,59.412565,86.470855
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,63.915158,100.000000,86.470855
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,68.899442,100.000000,86.470855
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,65.912930,87.564957,95.854986


Estrategia de trading con RSI y STOCHRSI

* El RSI y el STOCHRSI se pueden utilizar de diferentes maneras. En particular aquí se utilizará para comprar activos en un período muy específico (a la inversa de lo que se suele proponer).

 -- Cuando el indicador supere los 80 pptos se considerará una buena oportunidad para comprar hasta que vuelva a caer por debajo de esa marca, momento en el que se debería evaluar la venta del activo.

In [140]:
BTC_df_STOCHRSI_ET = BTC_df_RSI.copy()

In [141]:
BTC_df_STOCHRSI_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'RSI', 'STOCHRSI_fastk', 'STOCHRSI_fastd'],
      dtype='object')

In [142]:
# Se aplica estrategia de trading
STOCHRSI_ET_df = pd.DataFrame()
STOCHRSI_ET_df

STOCHRSI_ET_fastk = []

x=0
while x < len (BTC_df_STOCHRSI_ET ["Price"]): 
    
    if x<21: # Necesita 21 valores para empezar el proceso (por defecto del uso común del indicador)
        STOCHRSI_ET_fastk.append ("Mantener_posicion")
        
    elif BTC_df_STOCHRSI_ET ["STOCHRSI_fastk"] [x] >= 80:
        
        if BTC_df_STOCHRSI_ET ["STOCHRSI_fastk"] [x-1] >=80:
            STOCHRSI_ET_fastk.append ("Mantener_posicion")
            
        elif BTC_df_STOCHRSI_ET ["STOCHRSI_fastk"] [x-1] < 80:
            STOCHRSI_ET_fastk.append ("Comprar")
    
    elif BTC_df_STOCHRSI_ET ["STOCHRSI_fastk"] [x] < 80:
        
        if BTC_df_STOCHRSI_ET ["STOCHRSI_fastk"] [x-1] >= 80:
            STOCHRSI_ET_fastk.append ("Vender")
            
        if BTC_df_STOCHRSI_ET ["STOCHRSI_fastk"] [x-1] < 80:
            STOCHRSI_ET_fastk.append ("Mantener_posicion")
            
        
    x+=1
    
STOCHRSI_ET_df ["STOCHRSI_fastk_ET"] = pd.Series(STOCHRSI_ET_fastk)

In [143]:
STOCHRSI_ET_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   STOCHRSI_fastk_ET  4386 non-null   object
dtypes: object(1)
memory usage: 34.4+ KB


In [144]:
# Se aplica estrategia de trading
STOCHRSI_ET_fastd = []

x=0
while x < len (BTC_df_STOCHRSI_ET ["Price"]): 
    
    if x<21: # Necesita 21 valores para empezar el proceso (por defecto del uso común del indicador)
        STOCHRSI_ET_fastd.append ("Mantener_posicion")
        
    elif BTC_df_STOCHRSI_ET ["STOCHRSI_fastd"] [x] >= 80:
        
        if BTC_df_STOCHRSI_ET ["STOCHRSI_fastd"] [x-1] >=80:
            STOCHRSI_ET_fastd.append ("Mantener_posicion")
            
        elif BTC_df_STOCHRSI_ET ["STOCHRSI_fastd"] [x-1] < 80:
            STOCHRSI_ET_fastd.append ("Comprar")
    
    elif BTC_df_STOCHRSI_ET ["STOCHRSI_fastd"] [x] < 80:
        
        if BTC_df_STOCHRSI_ET ["STOCHRSI_fastd"] [x-1] >= 80:
            STOCHRSI_ET_fastd.append ("Vender")
            
        if BTC_df_STOCHRSI_ET ["STOCHRSI_fastd"] [x-1] < 80:
            STOCHRSI_ET_fastd.append ("Mantener_posicion")
            
        
    x+=1
    
STOCHRSI_ET_df ["STOCHRSI_fastd_ET"] = pd.Series(STOCHRSI_ET_fastd)

In [145]:
STOCHRSI_ET_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   STOCHRSI_fastk_ET  4386 non-null   object
 1   STOCHRSI_fastd_ET  4386 non-null   object
dtypes: object(2)
memory usage: 68.7+ KB


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_STOCHRSI_ET (ya posee los valores del indicador).

In [146]:
BTC_STOCHRSI_ET_df = pd.concat ([BTC_df_STOCHRSI_ET, STOCHRSI_ET_df], axis=1)

In [147]:
BTC_STOCHRSI_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,RSI,STOCHRSI_fastk,STOCHRSI_fastd,STOCHRSI_fastk_ET,STOCHRSI_fastd_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,54.341002,100.000000,100.000000,Mantener_posicion,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,49.615550,59.412565,86.470855,Vender,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,63.915158,100.000000,86.470855,Comprar,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,68.899442,100.000000,86.470855,Mantener_posicion,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,65.912930,87.564957,95.854986,Mantener_posicion,Mantener_posicion


#### Ultimate Oscillator (ULTOSC)

* Información (EN): https://www.investopedia.com/terms/u/ultimateoscillator.asp

In [148]:
# Se crea back up
BTC_df_ULTOSC = BTC_df.copy()

In [149]:
# Se calcula el indicador ULTOSC
BTC_df_ULTOSC ["ULTOSC"] = ta.ULTOSC(BTC_df_ULTOSC ["High"], BTC_df_ULTOSC ["Low"], BTC_df_ULTOSC ["Price"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

In [150]:
BTC_df_ULTOSC.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ULTOSC
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,55.402532
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,52.658819
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,61.793596
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,68.346203
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,64.588452


Observaciones:
- No se calculará una estrategia de trading.

#### Williams' %R (WILLR)

* Información (EN): https://www.investopedia.com/terms/w/williamsr.asp

In [151]:
# Se crea un back up
BTC_df_WILLR = BTC_df.copy()

In [152]:
# Se calcula Williams %R
BTC_df_WILLR ["WILLR"] = ta.WILLR(BTC_df_WILLR ["High"], BTC_df_WILLR ["Low"], BTC_df_WILLR ["Price"], timeperiod=14)

In [153]:
BTC_df_WILLR.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,WILLR
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,-34.498468
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,-47.925083
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,-5.012325
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,-7.208573
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,-12.631098


#### Agrupando todos los indicadores de momentum en un mismo data set

Data set sin estrategia de trading

In [154]:
BTC_df_ADX.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ADX', 'ADX_MDI', 'ADX_MDM', 'ADX_DX', 'ADX_PDI', 'ADX_PDM'],
      dtype='object')

In [155]:
BTC_df_ADXR.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ADXR'],
      dtype='object')

In [156]:
BTC_df_Aroon.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'Aroondown', 'Aroonup', 'Aroon_Oscillator'],
      dtype='object')

In [157]:
BTC_df_MACD.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'MACD', 'MACDSIGNAL', 'MACDHIST'],
      dtype='object')

In [158]:
BTC_df_MOM.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'MOM'],
      dtype='object')

In [159]:
BTC_df_ROC.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ROC'],
      dtype='object')

In [160]:
BTC_df_RSI.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'RSI', 'STOCHRSI_fastk', 'STOCHRSI_fastd'],
      dtype='object')

In [161]:
BTC_df_ULTOSC.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ULTOSC'],
      dtype='object')

In [162]:
BTC_df_WILLR.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'WILLR'],
      dtype='object')

In [163]:
BTC_ind_mom = pd.DataFrame ({'Date':BTC_df_ADX ["Date"], 'Price':BTC_df_ADX ["Price"], 
                             'Open': BTC_df_ADX ["Open"], 'High': BTC_df_ADX ["High"], 
                             'Low': BTC_df_ADX ["Low"], 'Vol.': BTC_df_ADX ["Vol."], 
                             'Percentage_diff': BTC_df_ADX ["Percentage_diff"],'Target': BTC_df_ADX ["Target"], 
                             'ADX': BTC_df_ADX ["ADX"], 'ADX_MDI': BTC_df_ADX ['ADX_MDI'],
                             'ADX_MDM': BTC_df_ADX ['ADX_MDI'],'ADX_DX': BTC_df_ADX ['ADX_DX'],
                             'ADX_PDI': BTC_df_ADX ['ADX_PDI'], 'ADX_PDM': BTC_df_ADX ['ADX_PDM'],
                             'Aroondown': BTC_df_Aroon['Aroondown'], 'Aroonup': BTC_df_Aroon['Aroonup'], 
                             'Aroon_Oscillator': BTC_df_Aroon['Aroon_Oscillator'],
                             'MACD': BTC_df_MACD ['MACD'], 'MACDSIGNAL': BTC_df_MACD ['MACDSIGNAL'], 
                             'MACDHIST': BTC_df_MACD ['MACDHIST'],
                             'MOM': BTC_df_MOM ['MOM'], 'ROC': BTC_df_ROC ['ROC'],
                             'RSI': BTC_df_RSI ['RSI'], 'STOCHRSI_fastk': BTC_df_RSI ['STOCHRSI_fastk'],
                             'STOCHRSI_fastd': BTC_df_RSI ['STOCHRSI_fastd'],
                             'ULTOSC': BTC_df_ULTOSC ['ULTOSC'], 'WILLR': BTC_df_WILLR ['WILLR']                            
                            })

In [164]:
BTC_ind_mom.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADX,ADX_MDI,ADX_MDM,ADX_DX,ADX_PDI,ADX_PDM,Aroondown,Aroonup,Aroon_Oscillator,MACD,MACDSIGNAL,MACDHIST,MOM,ROC,RSI,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,27.268083,24.194644,24.194644,6.885001,21.077646,3443.366972,7.142857,42.857143,35.714286,-636.592931,-960.502376,323.909445,648.8,0.031555,54.341002,100.000000,100.000000,55.402532,-34.498468
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,25.710035,22.840734,22.840734,5.455418,20.477541,3290.512188,0.000000,35.714286,35.714286,-577.232483,-883.848398,306.615914,-852.2,-0.039385,49.615550,59.412565,86.470855,52.658819,-47.925083
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,24.545625,20.207578,20.207578,9.408284,24.404841,4115.975603,64.285714,100.000000,35.714286,-385.327489,-784.144216,398.816727,914.6,0.042321,63.915158,100.000000,86.470855,61.793596,-5.012325
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,24.304487,17.742843,17.742843,21.169702,27.272446,4864.377346,57.142857,100.000000,42.857143,-160.032949,-659.321963,499.289013,1822.7,0.084433,68.899442,100.000000,86.470855,68.346203,-7.208573
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,24.080574,17.259314,17.259314,21.169702,26.529215,4516.921821,50.000000,92.857143,42.857143,-2.524886,-527.962547,525.437661,2301.7,0.110407,65.912930,87.564957,95.854986,64.588452,-12.631098


In [165]:
BTC_ind_mom.to_csv(r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_mom.csv", index=False, sep=";")

Data set con estrategias de trading

In [166]:
ADX_ET_df.columns

Index(['ADX_ET'], dtype='object')

In [167]:
ADXR_ET_df.columns

Index(['ADXR_ET'], dtype='object')

In [168]:
MACD_ET_df.columns

Index(['MACD_ET'], dtype='object')

In [169]:
ROC_ET_df.columns

Index(['ROC_ET'], dtype='object')

In [170]:
STOCHRSI_ET_df.columns

Index(['STOCHRSI_fastk_ET', 'STOCHRSI_fastd_ET'], dtype='object')

In [171]:
BTC_ind_mom_ET = pd.concat ([BTC_ind_mom, ADX_ET_df, ADXR_ET_df, MACD_ET_df, ROC_ET_df, STOCHRSI_ET_df], axis=1)

In [172]:
BTC_ind_mom_ET.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADX,ADX_MDI,ADX_MDM,ADX_DX,ADX_PDI,ADX_PDM,Aroondown,Aroonup,Aroon_Oscillator,MACD,MACDSIGNAL,MACDHIST,MOM,ROC,RSI,STOCHRSI_fastk,STOCHRSI_fastd,ULTOSC,WILLR,ADX_ET,ADXR_ET,MACD_ET,ROC_ET,STOCHRSI_fastk_ET,STOCHRSI_fastd_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,27.268083,24.194644,24.194644,6.885001,21.077646,3443.366972,7.142857,42.857143,35.714286,-636.592931,-960.502376,323.909445,648.8,0.031555,54.341002,100.000000,100.000000,55.402532,-34.498468,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,25.710035,22.840734,22.840734,5.455418,20.477541,3290.512188,0.000000,35.714286,35.714286,-577.232483,-883.848398,306.615914,-852.2,-0.039385,49.615550,59.412565,86.470855,52.658819,-47.925083,Mantener_posicion,Mantener_posicion,Mantener_posicion,Vender,Vender,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,24.545625,20.207578,20.207578,9.408284,24.404841,4115.975603,64.285714,100.000000,35.714286,-385.327489,-784.144216,398.816727,914.6,0.042321,63.915158,100.000000,86.470855,61.793596,-5.012325,Comprar,Mantener_posicion,Mantener_posicion,Comprar,Comprar,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,24.304487,17.742843,17.742843,21.169702,27.272446,4864.377346,57.142857,100.000000,42.857143,-160.032949,-659.321963,499.289013,1822.7,0.084433,68.899442,100.000000,86.470855,68.346203,-7.208573,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,24.080574,17.259314,17.259314,21.169702,26.529215,4516.921821,50.000000,92.857143,42.857143,-2.524886,-527.962547,525.437661,2301.7,0.110407,65.912930,87.564957,95.854986,64.588452,-12.631098,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion,Mantener_posicion


In [173]:
BTC_ind_mom_ET.to_csv(r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_mom_ET.csv", index=False, sep=";")

### Volumen indicators

#### Chaikin A/D Oscillator (ADOSC)

* Información (EN): https://www.investopedia.com/articles/active-trading/031914/understanding-chaikin-oscillator.asp

In [174]:
# Se crea un back up
BTC_df_ADOSC = BTC_df.copy()

In [175]:
# Se calcula el oscilador Chaiking A/D
BTC_df_ADOSC ["ADOSC"] = ta.ADOSC (BTC_df_ADOSC ["High"], BTC_df_ADOSC ["Low"], BTC_df_ADOSC ["Price"], BTC_df_ADOSC ["Vol."], fastperiod=3, slowperiod=10)

In [176]:
BTC_df_ADOSC.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADOSC
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,38511.511976
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,7776.544982
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,66185.614950
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,150988.804515
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,152531.617704


Estrategia de trading siguiendo el Chaikin A/D Oscillator (ADOSC)

* Se considera que si el indicador supera el valor 0 se debería evaluar la compra del activo (sobreventa).
* Se considera que si el indicador cae por debajo de 0 se debería evaluar la compra del activo (sobrecompra).
* En el resto de los casos se recomienda mantener la última posición (no hay grandes cambios).

In [178]:
BTC_df_ADOSC_ET = BTC_df_ADOSC.copy()

In [179]:
BTC_df_ADOSC_ET.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target', 'ADOSC'],
      dtype='object')

In [180]:
# Se aplica la estrategia de trading
ADOSC_ET_df = pd.DataFrame()
ADOSC_ET_df

ADOSC_ET = []

x=0
while x < len (BTC_df_ADOSC_ET ["Price"]): 
    
    if x<10: # Necesita 10 valores para empezar el proceso (por defecto del uso común del indicador)
        ADOSC_ET.append ("Mantener_posicion")
        
    elif BTC_df_ADOSC_ET ["ADOSC"] [x] >= 0:
        
        if BTC_df_ADOSC_ET ["ADOSC"] [x-1] >=0:
            ADOSC_ET.append ("Mantener_posicion")
            
        elif BTC_df_ADOSC_ET ["ADOSC"] [x-1] < 0:
            ADOSC_ET.append ("Comprar")
    
    elif BTC_df_ADOSC_ET ["ADOSC"] [x] < 0:
        
        if BTC_df_ADOSC_ET ["ADOSC"] [x-1] >= 0:
            ADOSC_ET.append ("Vender")
            
        if BTC_df_ADOSC_ET ["ADOSC"] [x-1] < 0:
            ADOSC_ET.append ("Mantener_posicion")
            
        
    x+=1
    
ADOSC_ET_df ["ADOSC_ET"] = pd.Series(ADOSC_ET)

In [181]:
ADOSC_ET_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ADOSC_ET  4386 non-null   object
dtypes: object(1)
memory usage: 34.4+ KB


Se concatena las columnas de la estrategia de trading a la tabla de BTC_df_ADOSC_ET (ya posee los valores del indicador).

In [182]:
BTC_ADOSC_ET_df = pd.concat ([BTC_df_ADOSC_ET, ADOSC_ET_df], axis=1)

In [183]:
BTC_ADOSC_ET_df.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADOSC,ADOSC_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,38511.511976,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,7776.544982,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,66185.614950,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,150988.804515,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,152531.617704,Mantener_posicion


#### Se crea los data set del indicador

Data set sin estrategia de trading

In [177]:
BTC_df_ADOSC.to_csv (r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_vol.csv", index=False, sep=";")

Data set con estrategia de trading

In [184]:
BTC_ind_vol_ET = pd.concat ([BTC_df_ADOSC, ADOSC_ET_df], axis=1)

In [185]:
BTC_ind_vol_ET.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ADOSC,ADOSC_ET
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,38511.511976,Mantener_posicion
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,7776.544982,Mantener_posicion
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,66185.614950,Mantener_posicion
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,150988.804515,Mantener_posicion
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,152531.617704,Mantener_posicion


In [187]:
BTC_ind_vol_ET.to_csv (r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_vol_ET.csv", index=False, sep=";")

### Cycle indicators
* Se calculan distintos elementos de los ciclos del precio (se analizará en detalle al momento de los análisis univariado, bivariado y multivariado).
* Información (EN). John F. Ehlers, Rocket Science for Traders. *Trading with signal processing methods*. Source code in EasyLanguage.
* Más información (EN): https://zorro-project.com/manual/en/ta.htm

#### Hilbert Transform - Dominant Cycle Period (HT_DCPERIOD), Hilbert Transform - Dominant Cycle Phase (HT_DCPHASE), Hilbert Transform - Phasor Components (HT_PHASOR), Hilbert Transform - SineWave (HT_SINE), Hilbert Transform - Trend vs Cycle Mode (HT_TRENDMODE)

In [188]:
# Se crea un back up
BTC_df_HT = BTC_df.copy ()

In [189]:
# Se calcula el HT_DCPERIOD
BTC_df_HT ["HT_DCPERIOD"] = ta.HT_DCPERIOD(BTC_df_HT ["Price"])

In [190]:
# Se calcula el HT_DCPHASE
BTC_df_HT ["HT_DCPHASE"] = ta.HT_DCPHASE(BTC_df_HT ["Price"])

In [191]:
# Se calcula el HT_PHASOR_inphase
BTC_df_HT ["HT_PHASOR_inphase"], BTC_df_HT ["HT_PHASOR_quadrature"] = ta.HT_PHASOR (BTC_df_HT ["Price"])

In [193]:
# Se calcula el HT_SINE_sine y el "HT_PHASOR_leadsine"
BTC_df_HT ["HT_SINE_sine"], BTC_df_HT ["HT_PHASOR_leadsine"] = ta.HT_SINE (BTC_df_HT ["Price"])

In [194]:
# Se calcula HT_PHASOR_integer
BTC_df_HT ["HT_SINE_integer"] = ta.HT_TRENDMODE (BTC_df_HT ["Price"])

In [195]:
BTC_df_HT.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,HT_DCPERIOD,HT_DCPHASE,HT_PHASOR_inphase,HT_PHASOR_quadrature,HT_SINE_sine,HT_PHASOR_leadsine,HT_SINE_integer
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,20.384732,188.760909,-1175.706860,-2381.833142,-0.152312,-0.806557,1
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,19.877096,208.648545,-1772.387604,25.302536,-0.479436,-0.959553,1
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,19.511967,217.212389,-1099.470256,2267.642942,-0.604771,-0.990777,1
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,19.143489,222.679567,59.865039,2323.609337,-0.677898,-0.999180,1
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,18.670726,170.267628,619.425710,1687.812158,0.169046,-0.577396,1


#### Se crea la base de datos

In [196]:
BTC_df_HT.to_csv (r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_cycle_indicators.csv", index=False, sep=";")

### Volatility indicators

#### Average True Range (ATR) 

* Información (EN): https://www.investopedia.com/terms/a/atr.asp

In [197]:
# Se crea un back up
BTC_df_ATR = BTC_df.copy()

In [198]:
# Se calcula el indicador ATR
BTC_df_ATR ["ATR"] = ta.ATR (BTC_df_ATR ["High"],BTC_df_ATR ["Low"],BTC_df_ATR ["Price"],timeperiod=14 )

In [199]:
BTC_df_ATR.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,ATR
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,1166.898729
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,1147.777391
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,1204.671863
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,1274.016730
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,1216.158392


#### Se crea la base de datos

In [200]:
BTC_df_ATR.to_csv (r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_ind_volat.csv", index=False, sep=";")

### Statistic Functions

#### Beta, CORREL
* Información (EN): https://www.investopedia.com/terms/b/beta.asp#:~:text=Beta%20Value%20Equal%20to%201.0,of%201.0%20has%20systematic%20risk.

#### Pearson's Correlation Coefficient (r)
* Información (EN): https://www.investopedia.com/terms/p/pearsoncoefficient.asp#:~:text=What%20Is%20the%20Pearson%20Coefficient,association%20between%20two%20continuous%20variables.

#### LINEARREG - Linear Regression
* Información (EN): https://www.investopedia.com/articles/trading/09/linear-regression-time-price.asp#:~:text=Linear%20regression%20is%20the%20analysis,stock%20is%20overbought%20or%20oversold.

#### LINEARREG_ANGLE - Linear Regression Angle
* Información (EN): https://www.mql5.com/en/market/product/19873?source=External%3Ahttps%3A%2F%2Fwww.google.com%2F

#### LINEARREG_INTERCEPT - Linear Regression Intercept
* Información (EN): https://www.investopedia.com/terms/r/regression.asp#:~:text=Linear%20regression%20is%20thus%20graphically,of%20the%20other%20is%20zero.

#### LINEARREG_SLOPE - Linear Regression Slope
* Información (EN): http://www.stat.yale.edu/Courses/1997-98/101/linreg.htm#:~:text=A%20linear%20regression%20line%20has,y%20when%20x%20%3D%200).

#### STDDEV - Standard Deviation
* Información (EN): https://www.investopedia.com/terms/s/standarddeviation.asp
* Información riesgo (EN): https://www.investopedia.com/ask/answers/021915/how-standard-deviation-used-determine-risk.asp

#### TSF - Time Series Forecast
* Información (EN): https://www.investopedia.com/terms/t/timeseries.asp#:~:text=Time%20series%20forecasting%20uses%20information,methods%2C%20success%20is%20not%20guaranteed.

#### VAR - Variance
* Información (EN): https://www.investopedia.com/terms/v/variance.asp

In [201]:
# Se crea un back up
BTC_df_SF= BTC_df.copy()

In [202]:
# Se calcula el valor Beta
BTC_df_SF ["Beta"] = ta.BETA(BTC_df_SF ["High"], BTC_df_SF ["Low"], timeperiod=5)

In [203]:
# Se calcula la correlación de Pearson
BTC_df_SF ["Pearson"] = ta.CORREL (BTC_df_SF ["High"], BTC_df_SF ["Low"], timeperiod=30)

In [204]:
# Se calcula el valor del precio dado por regresión lineal
BTC_df_SF ["Linearregression"] = ta.LINEARREG (BTC_df_SF ["Price"], timeperiod=14)

In [205]:
# Se calcula el ángulo de la regresión lineal
BTC_df_SF ["Linearangle"] = ta.LINEARREG_ANGLE (BTC_df_SF ["Price"], timeperiod=14)

In [206]:
# Se calcula la regresión lineal con intercepción
BTC_df_SF ["Linearintercept"] = ta.LINEARREG_INTERCEPT (BTC_df_SF ["Price"], timeperiod=14)

In [207]:
# Se calcula la curvatura de la regresión
BTC_df_SF ["Linearslope"] = ta.LINEARREG_SLOPE (BTC_df_SF ["Price"], timeperiod=14)

In [208]:
# Se calcula la varianza
BTC_df_SF ["VAR"] = ta.VAR (BTC_df_SF ["Price"], timeperiod=5, nbdev=1)

In [209]:
# Se calcula desviación estandar
BTC_df_SF ["STDDEV"] = ta.STDDEV (BTC_df_SF ["Price"], timeperiod=5, nbdev=1)

In [210]:
# Se calcula la predicción por time series forecasting method
BTC_df_SF ["TSF"] = ta.TSF (BTC_df_SF ["Price"], timeperiod=14)

In [211]:
BTC_df_SF.tail()

,Date,Price,Open,High,Low,Vol.,Percentage_diff,Target,Beta,Pearson,Linearregression,Linearangle,Linearintercept,Linearslope,VAR,STDDEV,TSF
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,0.925352,0.775534,20802.237143,88.315816,20360.105714,34.010110,4.057132e+05,636.956216,20836.247253
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,1.335584,0.780996,20673.194286,-64.096744,20699.962857,-2.059121,9.871333e+04,314.186781,20671.135165
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,-0.257208,0.831339,21078.882857,88.362015,20624.274286,34.969890,4.892406e+05,699.457371,21113.852747
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,0.015263,0.892832,21643.817143,89.338511,20517.854286,86.612527,1.087067e+06,1042.625182,21730.429670
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,-0.495034,0.907104,22101.408571,89.554376,20429.977143,128.571648,1.090209e+06,1044.130891,22229.980220


#### Se crea la base de datos

In [212]:
BTC_df_SF.to_csv (r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_statistic_functions.csv", index=False, sep=";")

### Patrones de comportamiento

* Se estudiarán patrones de comportamiento del precio ofrecidos por la librería ta-lib
* Información (EN): https://www.dailyfx.com/education/technical-analysis-chart-patterns

Se modifican las columnas del dataframe de BTC original para poder utilizarlo en una función automática que pasa todos los patrones de una sola vez.

In [213]:
# Se crea un back up
BTC_minor_letters = BTC_df.copy()

In [214]:
BTC_minor_letters.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Percentage_diff',
       'Target'],
      dtype='object')

In [215]:
# Se renombran las columnas
BTC_minor_letters_2 = BTC_minor_letters.rename (columns={'Date':'date', 'Price':'close', 'Open':'open', 'High':'high', 'Low':'low', 'Vol.':'vol', 'Porcentage_diff':'porcentage_diff',
       'Target':'target'})

In [216]:
BTC_minor_letters_2.columns

Index(['date', 'close', 'open', 'high', 'low', 'vol', 'Percentage_diff',
       'target'],
      dtype='object')

Se invocan las funciones de ta-lib para encontrar la correspondiente a los patrones

In [217]:
ta.get_function_groups().keys()

dict_keys(['Cycle Indicators', 'Math Operators', 'Math Transform', 'Momentum Indicators', 'Overlap Studies', 'Pattern Recognition', 'Price Transform', 'Statistic Functions', 'Volatility Indicators', 'Volume Indicators'])

In [218]:
ta.get_function_groups()

{'Cycle Indicators': ['HT_DCPERIOD',
  'HT_DCPHASE',
  'HT_PHASOR',
  'HT_SINE',
  'HT_TRENDMODE'],
 'Math Operators': ['ADD',
  'DIV',
  'MAX',
  'MAXINDEX',
  'MIN',
  'MININDEX',
  'MINMAX',
  'MINMAXINDEX',
  'MULT',
  'SUB',
  'SUM'],
 'Math Transform': ['ACOS',
  'ASIN',
  'ATAN',
  'CEIL',
  'COS',
  'COSH',
  'EXP',
  'FLOOR',
  'LN',
  'LOG10',
  'SIN',
  'SINH',
  'SQRT',
  'TAN',
  'TANH'],
 'Momentum Indicators': ['ADX',
  'ADXR',
  'APO',
  'AROON',
  'AROONOSC',
  'BOP',
  'CCI',
  'CMO',
  'DX',
  'MACD',
  'MACDEXT',
  'MACDFIX',
  'MFI',
  'MINUS_DI',
  'MINUS_DM',
  'MOM',
  'PLUS_DI',
  'PLUS_DM',
  'PPO',
  'ROC',
  'ROCP',
  'ROCR',
  'ROCR100',
  'RSI',
  'STOCH',
  'STOCHF',
  'STOCHRSI',
  'TRIX',
  'ULTOSC',
  'WILLR'],
 'Overlap Studies': ['BBANDS',
  'DEMA',
  'EMA',
  'HT_TRENDLINE',
  'KAMA',
  'MA',
  'MAMA',
  'MAVP',
  'MIDPOINT',
  'MIDPRICE',
  'SAR',
  'SAREXT',
  'SMA',
  'T3',
  'TEMA',
  'TRIMA',
  'WMA'],
 'Pattern Recognition': ['CDL2CROWS',
  'C

Se construye una lista de con los patrones que reconoce ta-lib y se aplica "abstract" (aplica toda una lista a los datos del dataframe que se le pase)

In [219]:
# Se pasan todos los patrones
all_indicators_patrons = [method for method in dir(abstract) if method.startswith('CDL')]

In [220]:
# Se calculan todos los patrones
df_indicator = pd.DataFrame()
for indicator in all_indicators_patrons:
    df_indicator[str(indicator)] = getattr(abstract, indicator)(BTC_minor_letters_2)

In [221]:
df_indicator.tail()

,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
4381,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4382,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100
4383,0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4384,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4385,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-100,0,0,0,0,0,0,0,0,0,0


In [222]:
df_indicator.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 61 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   CDL2CROWS            4386 non-null   int32
 1   CDL3BLACKCROWS       4386 non-null   int32
 2   CDL3INSIDE           4386 non-null   int32
 3   CDL3LINESTRIKE       4386 non-null   int32
 4   CDL3OUTSIDE          4386 non-null   int32
 5   CDL3STARSINSOUTH     4386 non-null   int32
 6   CDL3WHITESOLDIERS    4386 non-null   int32
 7   CDLABANDONEDBABY     4386 non-null   int32
 8   CDLADVANCEBLOCK      4386 non-null   int32
 9   CDLBELTHOLD          4386 non-null   int32
 10  CDLBREAKAWAY         4386 non-null   int32
 11  CDLCLOSINGMARUBOZU   4386 non-null   int32
 12  CDLCONCEALBABYSWALL  4386 non-null   int32
 13  CDLCOUNTERATTACK     4386 non-null   int32
 14  CDLDARKCLOUDCOVER    4386 non-null   int32
 15  CDLDOJI              4386 non-null   int32
 16  CDLDOJISTAR          438

Observaciones:
- No hay valores nulos.

In [223]:
df_indicator_values = df_indicator.columns

In [224]:
df_indicator_values

Index(['CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE',
       'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS',
       'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY',
       'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK',
       'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI',
       'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR',
       'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER',
       'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS', 'CDLHIGHWAVE',
       'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS',
       'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH',
       'CDLLADDERBOTTOM', 'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU',
       'CDLMATCHINGLOW', 'CDLMATHOLD', 'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR',
       'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN', 'CDLRISEFALL3METHODS',
       'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSHORTLI

In [225]:
# Se imprime los valores correspondientes a cada patrón
for col in df_indicator_values:
    values = df_indicator [col].value_counts()
    print (col, values)
    print ("------*------")

CDL2CROWS 0    4386
Name: CDL2CROWS, dtype: int64
------*------
CDL3BLACKCROWS 0    4386
Name: CDL3BLACKCROWS, dtype: int64
------*------
CDL3INSIDE  0      4377
-100       5
 100       4
Name: CDL3INSIDE, dtype: int64
------*------
CDL3LINESTRIKE  0      4381
-100       4
 100       1
Name: CDL3LINESTRIKE, dtype: int64
------*------
CDL3OUTSIDE  0      4294
-100      46
 100      46
Name: CDL3OUTSIDE, dtype: int64
------*------
CDL3STARSINSOUTH 0    4386
Name: CDL3STARSINSOUTH, dtype: int64
------*------
CDL3WHITESOLDIERS 0      4383
100       3
Name: CDL3WHITESOLDIERS, dtype: int64
------*------
CDLABANDONEDBABY 0    4386
Name: CDLABANDONEDBABY, dtype: int64
------*------
CDLADVANCEBLOCK  0      4325
-100      61
Name: CDLADVANCEBLOCK, dtype: int64
------*------
CDLBELTHOLD  0      3687
 100     385
-100     314
Name: CDLBELTHOLD, dtype: int64
------*------
CDLBREAKAWAY 0    4386
Name: CDLBREAKAWAY, dtype: int64
------*------
CDLCLOSINGMARUBOZU  0      3962
 100     278
-100     146


Se concatena el dataframe de precios de BTC con el conjunto de patrones invocados

In [226]:
BTC_df_patterns_ind = pd.concat ([BTC_minor_letters_2, df_indicator], axis=1) 

In [227]:
BTC_df_patterns_ind.tail()

,date,close,open,high,low,vol,Percentage_diff,target,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
4381,2022-07-16,21209.9,20825.2,21561.3,20484.4,136890.0,1.85,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4382,2022-07-17,20785.6,21209.8,21654.4,20755.2,132810.0,-2.00,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100
4383,2022-07-18,22525.8,20785.6,22714.9,20770.6,279720.0,8.37,1.0,0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4384,2022-07-19,23410.2,22529.3,23757.3,21581.8,308910.0,3.93,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4385,2022-07-20,23149.1,23412.0,23429.9,22965.9,290210.0,-1.12,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-100,0,0,0,0,0,0,0,0,0,0


In [228]:
BTC_df_patterns_ind.to_csv (r"D:\Luciano\Programación\Data science\Trabajo final\Bases de datos para modelos\BTC_patterns_indicator.csv", index=False, sep=";")

# Bases de datos resultantes (diccionario de archivos .csv)

**BTC tendencia (sin estrategia de trading) = BTC_ind_trend.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **SMA_7d, SMA_15d, SMA_30d, SMA_50d, SMA_100d, SMA_200d**: media móvil simple (diferentes períodos) (USD) (float) (Variable numérica continua)
- **EMA_7d, EMA_15d, EMA_30d, EMA_50d, EMA_100d, EMA_200d**: media móvil exponencial (diferentes períodos) (USD) (float) (Variable numérica continua)
- **HT_TRENDLINE**: indicador de Hilbert Transform - Instantaneous Trendline (USD) (float) (Variable numérica continua)
- **BBANDS_upper_band**: banda superior del indicador Bandas de Bollinger (USD) (float) (Variable numérica continua)
- **BBANDS_middle_band**: banda media del indicador Bandas de Bollinger (USD) (float) (Variable numérica continua)
- **BBANDS_lower_band**: banda inferior del indicador Bandas de Bollinger (USD) (float) (Variable numérica continua)
- **PSAR**: indicador Parabolic SAR (USD) (float) (Variable numérica continua)

**BTC tendencia (con estrategia de trading) = BTC_ind_trend_ET.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **SMA_7d, SMA_15d, SMA_30d, SMA_50d, SMA_100d, SMA_200d**: media móvil simple (diferentes períodos) (USD) (float) (Variable numérica continua)
- **EMA_7d, EMA_15d, EMA_30d, EMA_50d, EMA_100d, EMA_200d**: media móvil exponencial (diferentes períodos) (USD) (float) (Variable numérica continua)
- **HT_TRENDLINE**: indicador de Hilbert Transform - Instantaneous Trendline (USD) (float) (Variable numérica continua)
- **BBANDS_upper_band**: banda superior del indicador Bandas de Bollinger (USD) (float) (Variable numérica continua)
- **BBANDS_middle_band**: banda media del indicador Bandas de Bollinger (USD) (float) (Variable numérica continua)
- **BBANDS_lower_band**: banda inferior del indicador Bandas de Bollinger (USD) (float) (Variable numérica continua)
- **PSAR**: indicador Parabolic SAR (USD) (float) (Variable numérica continua)

Estrategia de trading = cuando el valor de la media queda por debajo del precio se compra (porque hay tendencia a subir) y cuando queda por debajo se vende (porque hay tendencia a bajar).
- **SMA_7d_ET, SMA_15d_ET, SMA_30d_ET, SMA_50d_ET, SMA_100d_ET, SMA_200d_ET**: valores categóricos de la aplicación de la estrategia de trading SMA (comprar, vender, mantener posición) (str) (Variable categórica)
- **EMA_7d_ET, EMA_15d_ET, EMA_30d_ET, EMA_50d_ET, EMA_100d_ET, EMA_200d_ET**: valores categóricos de la aplicación de la estrategia de trading EMA (comprar, vender, mantener posición) (str) (Variable categórica)
- **HT_TRENDLINE_ET**: valores categóricos de la aplicación de la estrategia de trading HT Trendline (comprar, vender, mantener posición) (str) (Variable categórica)
- **PSAR_ET**: valores categóricos de la aplicación de la estrategia de trading PSAR (comprar, vender, mantener posición) (str) (Variable categórica)

**BTC indicadores de momentum (sin estrategias de trading) = BTC_ind_mom.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **ADX**: valores del índice de movimiento direccional medio (float) (Variable numérica continua)
- **ADX_MDI**: valores del Corriente Menos Movimiento Direccional (MDI) (float) (Variable numérica continua)
- **ADX_MDM**: valores del Mínimo Actual Menos el Mínimo Anterior (-DM) (float) (Variable numérica continua)
- **ADX_DX**: valores del índice de movimiento direccional (DX) (float) (Variable numérica continua)
- **ADX_PDI**: valores del Corriente Plus Movimiento Direccional (PDM) (float) (Variable numérica continua)
- **ADX_PDM**: valores del Máximo Actual Menos el Máximo Anterior (+DM) (float) (Variable numérica continua)
- **Aroondown**: valores del número de períodos desde los últimos 25 períodos máximos (float) (Variable numérica continua)
- **Aroonup**: valores del número de períodos desde los últimos 25 períodos máximos (float) (Variable numérica continua)
- **Aroon_Oscillator**: valores del Oscilador Aroon (float) (Variable numérica continua)
- **MACD**: valores del MACD (float) (Variable numérica continua)
- **MACDSIGNAL**: valores de la señal de MACD (float) (Variable numérica continua)
- **MACDHIST**: valores del histograma del MACD (float) (Variable numérica continua)
- **MOM**: valores del indicador Momentum (float) (Variable numérica continua)
- **ROC**: valores de la tasa de cambio de una variable durante un período específico de tiempo (float) (Variable numérica continua)
- **RSI**: valores del índice de fuerza relativa (float) (Variable numérica continua)
- **STOCHRSI_fastk**: valores de la aplicación del método estocástico al indicador RSI (con variable d) (más lento) (float) (Variable numérica continua)
- **STOCHRSI_fastd**: valores de la aplicación del método estocástico al indicador RSI (con variable k) (más rápido) (float) (Variable numérica continua)
- **ULTOSC**: valores del indicador Ultimate Oscillator (float) (Variable numérica continua)
- **WILLR**: valores del indicador Williams %R (float) (Variable numérica continua)

**BTC indicadores de momentum (con estrategias de trading) = BTC_ind_mom_ET.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **ADX**: valores del índice de movimiento direccional medio (float) (Variable numérica continua)
- **ADX_MDI**: valores del Corriente Menos Movimiento Direccional (MDI) (float) (Variable numérica continua)
- **ADX_MDM**: valores del Mínimo Actual Menos el Mínimo Anterior (-DM) (float) (Variable numérica continua)
- **ADX_DX**: valores del índice de movimiento direccional (DX) (float) (Variable numérica continua)
- **ADX_PDI**: valores del Corriente Plus Movimiento Direccional (PDM) (float) (Variable numérica continua)
- **ADX_PDM**: valores del Máximo Actual Menos el Máximo Anterior (+DM) (float) (Variable numérica continua)
- **Aroondown**: valores del número de períodos desde los últimos 25 períodos máximos (float) (Variable numérica continua)
- **Aroonup**: valores del número de períodos desde los últimos 25 períodos máximos (float) (Variable numérica continua)
- **Aroon_Oscillator**: valores del Oscilador Aroon (float) (Variable numérica continua)
- **MACD**: valores del MACD (float) (Variable numérica continua)
- **MACDSIGNAL**: valores de la señal de MACD (float) (Variable numérica continua)
- **MACDHIST**: valores del histograma del MACD (float) (Variable numérica continua)
- **MOM**: valores del indicador Momentum (float) (Variable numérica continua)
- **ROC**: valores de la tasa de cambio de una variable durante un período específico de tiempo (float) (Variable numérica continua)
- **RSI**: valores del índice de fuerza relativa (float) (Variable numérica continua)
- **STOCHRSI_fastk**: valores de la aplicación del método estocástico al indicador RSI (con variable d) (más lento) (float) (Variable numérica continua)
- **STOCHRSI_fastd**: valores de la aplicación del método estocástico al indicador RSI (con variable k) (más rápido) (float) (Variable numérica continua)
- **ULTOSC**: valores del indicador Ultimate Oscillator (float) (Variable numérica continua)
- **WILLR**: valores del indicador Williams %R (float) (Variable numérica continua)

Las estrategias son explicadas oportunamente al momento de calcularse (varían los rangos de compra y venta según indicador).
- **ADX_ET**: valores categóricos de la aplicación de la estrategia ADX (comprar, vender, mantener posición) (str)
- **ADXR_ET**: valores categóricos de la aplicación de la estrategia ADXR (comprar, vender, mantener posición) (str)
- **MACD_ET**: valores categóricos de la aplicación de la estrategia MACD (comprar, vender, mantener posición) (str)
- **ROC_ET**: valores categóricos de la aplicación de la estrategia ROC (comprar, vender, mantener posición) (str)
- **STOCHRSI_fastk_ET**: valores categóricos de la aplicación de la estrategia STOCHRSI_fastk (comprar, vender, mantener posición) (str)
- **STOCHRSI_fastd_ET**: valores categóricos de la aplicación de la estrategia STOCHRSI_fastd (comprar, vender, mantener posición) (str)

**BTC indicadores de volumen (sin estrategias de trading) = BTC_ind_vol.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **ADOSC**: indica la fuerza de los movimientos de precios y la presión subyacente de compra y venta (float) (Variable numérica continua)

**BTC Indicadores de ciclo = BTC_ind_cycle_indicators.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **HT_DCPERIOD**: Hilbert Transform Dominant Cycle Period (período de tiempo del ciclo dominante actual en la serie de Datos, en barras; rango válido = 10..60 barras, retraso = ~10 barras) (float) (Variable numérica continua).
- **HT_DCPHASE**: Hilbert Transform - Dominant Cycle Phase (fase del ciclo dominante en una barra específica entre 0 a 360 grados dentro del período de transformada de Hilbert actual medido instantáneamente en esa barra) (grados) (float) (Variable numérica continua)
- **HT_PHASOR_inphase**: el elemento inphase de la amplitud de la sinusoide modulada (float) (Variable numérica continua)
- **HT_PHASOR_quadrature**: el elemento quadrature de la amplitud de la sinusoide modulada (float) (Variable numérica continua)
- **HT_SINE_sine**: Hilbert Transform - SineWave (-1 a 1) (float) (Variable numérica continua)
- **HT_PHASOR_leadsine**: Hilbert Transform seno del lead (-1 a 1) (float) (Variable numérica continua)
- **HT_SINE_integer**: Hilbert Transform trend indicator (retorna 1 para Trend Mode, 0 para Cycle Mode) (int) (Funciona como variable categórica)

**BTC Indicadores de volatilidad = BTC_ind_volat.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **ATR**: valores del indicador Average True Range (Promedio de Rango Verdadero) (float) (Variable numérica continua)

**BTC funciones estadísticas = BTC_statistic_functions.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **Beta**: valores beta de correlación (-1 a 1) (float) (Variable numérica continua)
- **Pearson**: valores de correlación de Pearson (-1 a 1) (float) (Variable numérica continua)
- **Linearregression**: precio estimado por regresión lineal (USD) (float) (Variable numérica continua)
- **Linearangle**: ángulo de la mejor línea de regressión (ángulos) (float) (Variable numérica continua)
- **Linearintercept**: intercepción de la regresión lineal (USD) (float) (Variable numérica continua)
- **Linearslope**: curvatura de la regresión lineal (grados) (float) (Variable numérica continua)
- **VAR**: varianza (medida de dispersión definida como la esperanza del cuadrado de la desviación de dicha variable respecto a su media) (float) (Variable numérica continua)
- **STDDEV**: desviación estándar (medición de la dispersión de los datos) (float) (Variable numérica continua)
- **TSF**: Time Series Forecasting (predicción de precio) (USD) (float) (Variable numérica continua)

**BTC patrones del precio = BTC_patterns_indicator.csv**

- **Date**: fecha en YYYY-MM-DD (object)
- **Price**: precio de cierre de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Open**: precio de apertura de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **High**: precio más alto de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Low**: precio más bajo de BTC en el día de la fecha (USD) (float) (Variable numérica continua)
- **Vol.**: volumen de BTC (Número de intercambios) en el día de la fecha (float) (Variable numérica continua)
- **Percentage_diff**: diferencia porcentual del precio de BTC en la fecha [x+1] con respecto a la fecha [x] (float) (Variable numérica continua) 
- **Target**: 1 indica que en el día de la fecha el precio de BTC subió, y 0 que el precio bajó (float) (Se la tratará como variable categórica)


- **CDL2CROWS**: patrón 2 crows (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDL3BLACKCROWS**: patrón 3 black crows (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDL3INSIDE**: patrón 3 inside (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDL3LINESTRIKE**: patrón three lines strike (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDL3OUTSIDE**: patrón three outside (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDL3STARSINSOUTH**: patrón 3 stars in south (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDL3WHITESOLDIERS**: patrón 3 white soldiers (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLABANDONEDBABY**: patrón abandone baby (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLADVANCEBLOCK**: patrón advance block (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLBELTHOLD**: patrón belt hold (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLBREAKAWAY**: patrón break away (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLCLOSINGMARUBOZU**: patrón closing Marubozu (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLCONCEALBABYSWALL**: patrón conceling baby swallow (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLCOUNTERATTACK**: patrón counter attack (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLDARKCLOUDCOVER**: patrón dark cloud cover (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLDOJI**: patrón doji (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLDOJISTAR**: patrón doji star (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLDRAGONFLYDOJI**: patrón dragonfly doji (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLENGULFING**: patrón engulfing (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLEVENINGDOJISTAR**: patrón evening doji star (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLEVENINGSTAR**: patrón evening star (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLGAPSIDESIDEWHITE**: patrón gap side-by-side (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLGRAVESTONEDOJI**: patrón gravestone doji (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLHAMMER**: patrón hammer (hacia abajo/arriba) (0, 100) (int) (Funcionan como variable categórica)
- **CDLHANGINGMAN**: patrón hanging man (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLHARAMI**: patrón harami (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLHARAMICROSS**: patrón harami cross (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLHIGHWAVE**: patrón high wave (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLHIKKAKE**: patrón Hikkake (hacia abajo/arriba) (-200, 0, 200) (int) (Funcionan como variable categórica)
- **CDLHIKKAKEMOD**: patrón Hikkake moderated (hacia abajo/arriba) (-200, 0, 200) (int) (Funcionan como variable categórica)
- **CDLHOMINGPIGEON**: patrón homing pigeon (hacia abajo/arriba) (-200, 0, 200) (int) (Funcionan como variable categórica)
- **CDLIDENTICAL3CROWS**: patrón identical three crows (hacia abajo/arriba) (-200, 0, 200) (int) (Funcionan como variable categórica)
- **CDLINNECK**: patrón in neck (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLINVERTEDHAMMER**: patrón inverted hammer (hacia abajo/arriba) (0, 100) (int) (Funcionan como variable categórica)
- **CDLKICKING**: patrón kicking (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLKICKINGBYLENGTH**: patrón kicking by length (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLLADDERBOTTOM**: patrón ladder bottom (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLLONGLEGGEDDOJI**: patrón long-legged doji (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLLONGLINE**: patrón long line (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLMARUBOZU**: patrón Marubozu (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLMATCHINGLOW**: patrón matching lows (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLMATHOLD**: patrón Matt hold (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLMORNINGDOJISTAR**: patrón morning doji star (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLMORNINGSTAR**: patrón morning star (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLONNECK**: patrón on-neck (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLPIERCING**: patrón piercing (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLRICKSHAWMAN**: patrón Rickshaw man (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLRISEFALL3METHODS**: falling and rising 3 method (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLSEPARATINGLINES**: patrón separating lines (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLSHOOTINGSTAR**: patrón shooting star (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLSHORTLINE**: patrón shortline (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLSPINNINGTOP**: patrón spinning top (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLSTALLEDPATTERN**: patrón stalled (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLSTICKSANDWICH**: patrón sandwich (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLTAKURI**: patrón Takuri (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLTASUKIGAP**: patrón Tasuki gap (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLTHRUSTING**: patrón Thrusting lines (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLTRISTAR**: patrón tri-star (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLUNIQUE3RIVER**: patrón unique three river (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLUPSIDEGAP2CROWS**: patrón downside/upside two crows (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)
- **CDLXSIDEGAP3METHODS**: downside/upside three methods (hacia abajo/arriba) (-100, 0, 100) (int) (Funcionan como variable categórica)